<a href="https://colab.research.google.com/github/Anurag07264/2022-web-developers/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
#!unzip archive.zip

In [ ]:
import os
import cv2

# Define the base directory of the dataset
base_dir = './isl_data_grey_split/train'

# Define the labels for each category
labels = [str(i) for i in range(0, 10)] + [chr(i) for i in range(ord('a'), ord('z')+1)]

# Create a dictionary to map labels to numerical values
label_dict = {label: i for i, label in enumerate(labels)}

# Initialize an empty list to store the data
data = []

# Loop over each label in the current category
for label in labels:
    # Define the path to the current label
    path = os.path.join(base_dir, label)
    # Loop over each image in the current label directory
    for img in os.listdir(path):
        # Only process PNG images
        if img.endswith('.png'):
            # Define the path to the current image
            img_path = os.path.join(path, img)
            # Read the image
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            # Resize the image to 64x64 pixels
            image = cv2.resize(image, (64, 64))
            # Append the image and its numerical label to the data list
            data.append([image, label_dict[label]])

# At this point, the 'data' list contains all preprocessed images and their corresponding numerical labels
print(data)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming 'data' is your preprocessed data
images, labels = zip(*data)

# Convert the list to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Normalize the images
images = images / 255.0

# Split the data into a training set and a test set
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Reshape the data to fit the model
train_images = train_images.reshape(-1, 64, 64, 1)
test_images = test_images.reshape(-1, 64, 64, 1)

# Create the model
model = Sequential()

# Add the layers
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(np.unique(labels)), activation='softmax'))  # The number of nodes in the last layer should match the number of classes

# Compile the model
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, validation_data=(test_images, test_labels), epochs=10)

# Save the model
model.save('my_model.h5')


Epoch 1/10
488/488 [==============================] - 85s 172ms/step - loss: 1.0205 - accuracy: 0.7105 - val_loss: 0.0616 - val_accuracy: 0.9861
Epoch 2/10
488/488 [==============================] - 91s 186ms/step - loss: 0.2910 - accuracy: 0.9009 - val_loss: 0.0340 - val_accuracy: 0.9918
Epoch 3/10
488/488 [==============================] - 86s 176ms/step - loss: 0.2265 - accuracy: 0.9198 - val_loss: 0.0212 - val_accuracy: 0.9938
Epoch 4/10
488/488 [==============================] - 86s 176ms/step - loss: 0.1697 - accuracy: 0.9387 - val_loss: 0.0194 - val_accuracy: 0.9944
Epoch 5/10
488/488 [==============================] - 92s 188ms/step - loss: 0.1377 - accuracy: 0.9484 - val_loss: 0.0157 - val_accuracy: 0.9959
Epoch 6/10
488/488 [==============================] - 86s 177ms/step - loss: 0.1274 - accuracy: 0.9544 - val_loss: 0.0125 - val_accuracy: 0.9969
Epoch 7/10
488/488 [==============================] - 88s 180ms/step - loss: 0.1149 - accuracy: 0.9559 - val_loss: 0.0100 - val_ac

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import os
import cv2

# Define the base directory of the test dataset
base_dir = './isl_data_grey_split/test/'
# Define the labels for each category
labels = [str(i) for i in range(0, 10)] + [chr(i) for i in range(ord('a'), ord('z')+1)]

# Create a dictionary to map labels to numerical values
label_dict = {label: i for i, label in enumerate(labels)}

# Initialize an empty list to store the test data
test_data = []

# Loop over each label in the current category
for label in labels:
    # Define the path to the current label
    path = os.path.join(base_dir, label)
    # Loop over each image in the current label directory
    for img in os.listdir(path):
        # Only process PNG images
        if img.endswith('.png'):
            # Define the path to the current image
            img_path = os.path.join(path, img)
            # Read the image
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            # Resize the image to 64x64 pixels
            image = cv2.resize(image, (64, 64))
            # Append the image and its numerical label to the test data list
            test_data.append([image, label_dict[label]])

# At this point, the 'test_data' list contains all preprocessed test images and their corresponding numerical labels

# Load the model
model = load_model('my_model.h5')

# Separate the test images and labels
test_images, test_labels = zip(*test_data)

# Convert the list to numpy arrays
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Normalize the test images
test_images = test_images / 255.0

# Reshape the test images to fit the model
test_images = test_images.reshape(-1, 64, 64, 1)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_images, test_labels)

print(f'The accuracy of the model on the test set is {accuracy * 100:.2f}%')


153/153 [==============================] - 7s 45ms/step - loss: 0.0068 - accuracy: 0.9988
The accuracy of the model on the test set is 99.88%


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the model
model = load_model('my_model.h5')

# Start the webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    if not ret:
        break

    # Convert the image to grayscale
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Resize the image to 64x64 pixels
    image = cv2.resize(image, (64, 64))

    # Normalize the image
    image = image / 255.0

    # Reshape the image to fit the model
    image = image.reshape(1, 64, 64, 1)

    # Make a prediction
    prediction = model.predict(image)

    # Convert the prediction to a label
    label = np.argmax(prediction, axis=1)

    # Display the label on the frame
    cv2.putText(frame, str(label), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the resulting frame
    cv2.imshow('Sign Language Recognition', frame)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()


In [ ]:
!pip install flask-ngrok


In [ ]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the model
model = load_model('my_model.h5')

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when the app is run

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        # Get the file from the POST request
        file = request.files['file']

        # Save the file to ./uploads
        file.save('uploads/image.png')

        # Load the image
        image = cv2.imread('uploads/image.png', cv2.IMREAD_GRAYSCALE)

        # Preprocess the image
        image = cv2.resize(image, (64, 64))
        image = image / 255.0
        image = image.reshape(1, 64, 64, 1)

        # Make a prediction
        prediction = model.predict(image)

        # Convert the prediction to a label
        label = np.argmax(prediction, axis=1)

        # Return the label
        return str(label)

    return render_template('index.html')

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://8312-35-245-232-222.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [24/Nov/2023 10:40:04] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2023 10:40:05] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
html = """
<!DOCTYPE html>
<html>
<body>

<h2>Upload File</h2>
<form action="/" method="post" enctype="multipart/form-data">
  Select image to upload:
  <input type="file" name="file" id="file">
  <input type="submit" value="Upload Image" name="submit">
</form>

</body>
</html>
"""

# Write the HTML content to 'templates/index.html'
os.makedirs('templates', exist_ok=True)
with open('templates/index.html', 'w') as f:
    f.write(html)
